In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
import tensorflow as tf

In [61]:
!pip install lime

     |████████████████████████████████| 276kB 2.6MB/s eta 0:00:01
     |████████████████████████████████| 3.7MB 37.3MB/s eta 0:00:01
  Created wheel for lime: filename=lime-0.2.0.0-cp36-none-any.whl size=284181 sha256=f1907c1a9c6388ad9f32069e5906c5aacf7fe7f90c0cb79eea1557729dec6959
  Stored in directory: /Users/kyledecember1/Library/Caches/pip/wheels/22/f2/ec/e5ebd07348b2b1ac722e91c2f549fcc220f7d5f25497a61232
Successfully built lime
  Found existing installation: Pillow 6.1.0
    Uninstalling Pillow-6.1.0:
      Successfully uninstalled Pillow-6.1.0


In [16]:
datagen = keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255)

train_path = os.path.join(os.pardir, os.pardir, 'data/train')
final_test_path = os.path.join(os.pardir, os.pardir, 'data/val')
test_path = os.path.join(os.pardir, os.pardir, 'data/test')

In [ ]:
image_size = 32
batch_size = 32

train_generator = datagen.flow_from_directory(
                train_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

test_generator = datagen.flow_from_directory(
                test_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

## Test known model to ensure functionality

In [ ]:
IMG_SHAPE = (image_size, image_size, 3)

base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
base_model.trainable = False

In [ ]:
model1 = tf.keras.Sequential([
  base_model,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model1.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
epochs = 10
steps_per_epoch = train_generator.n // batch_size
validation_steps = test_generator.n // batch_size

history = model1.fit_generator(train_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs,
                              workers=4,
                              validation_data=test_generator,
                              validation_steps=validation_steps)

## Self-Made Model

In [ ]:
# create some convolutional layers, along with basic dropout for regularization 

model2 = tf.keras.Sequential()

model2.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Dropout(.5))
model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Dropout(.3))
model2.add(layers.Conv2D(64, (3, 3), activation='relu'))

model2.add(layers.Flatten())
model2.add(layers.Dense(64, activation='relu'))
model2.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model2.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model2.fit_generator(train_generator,
                     steps_per_epoch = steps_per_epoch,
                     epochs=epochs,
                     workers=4,
                     validation_data=test_generator,
                     validation_steps=validation_steps)

## Resizing images to 160x160


In [ ]:
image_size = 160
batch_size = 32


train_generator160 = datagen.flow_from_directory(
                train_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

test_generator160 = datagen.flow_from_directory(
                test_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')


In [ ]:
# create some convolutional layers, along with basic dropout for regularization, 

model3 = tf.keras.Sequential()

model3.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)))
model3.add(layers.MaxPooling2D((2, 2)))
model3.add(layers.Dropout(.5))
model3.add(layers.Conv2D(64, (3, 3), activation='relu'))
model3.add(layers.MaxPooling2D((2, 2)))
model3.add(layers.Dropout(.3))
model3.add(layers.Conv2D(64, (3, 3), activation='relu'))

model3.add(layers.Flatten())
model3.add(layers.Dense(64, activation='relu'))
model3.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
epochs = 10
steps_per_epoch160 = train_generator160.n // batch_size
validation_steps160 = test_generator160.n // batch_size

model3.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model3.fit_generator(train_generator160,
                     steps_per_epoch = steps_per_epoch160,
                     epochs=epochs,
                     workers=4,
                     validation_data=test_generator160,
                     validation_steps=validation_steps160)

analysis: variant accuracy indicated dropout may be too high early in network, unclear if issue is overfit or dropout coincidence

solve: remove dropout entirely, then add backwards (from output layer to input layer), until stable and accurate

## Removing Regularization


In [ ]:
image_size = 160
batch_size = 32

train_generator160 = datagen.flow_from_directory(
                train_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

test_generator160 = datagen.flow_from_directory(
                test_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')



In [ ]:
# create some convolutional layers, along with basic dropout for regularization, 

model4 = tf.keras.Sequential()

model4.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)))
model4.add(layers.MaxPooling2D((2, 2)))
model4.add(layers.Conv2D(64, (3, 3), activation='relu'))
model4.add(layers.MaxPooling2D((2, 2)))
model4.add(layers.Conv2D(64, (3, 3), activation='relu'))

model4.add(layers.Flatten())
model4.add(layers.Dense(64, activation='relu'))
model4.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
epochs = 10
steps_per_epoch160 = train_generator160.n // batch_size
validation_steps160 = test_generator160.n // batch_size

model4.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model4.fit_generator(train_generator160,
                     steps_per_epoch = steps_per_epoch160,
                     epochs=epochs,
                     workers=4,
                     validation_data=test_generator160,
                     validation_steps=validation_steps160)

##  increasing batch size

In [ ]:
image_size = 160
batch_size64 = 64

train_generator160 = datagen.flow_from_directory(
                train_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size64,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

test_generator160 = datagen.flow_from_directory(
                test_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size64,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')


In [ ]:
model5 = tf.keras.Sequential()

model5.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)))
model5.add(layers.MaxPooling2D((2, 2)))

model5.add(layers.Conv2D(64, (3, 3), activation='relu'))
model5.add(layers.MaxPooling2D((2, 2)))

model5.add(layers.Conv2D(64, (3, 3), activation='relu'))
model5.add(layers.MaxPooling2D((2, 2)))

model5.add(layers.Flatten())
model5.add(layers.Dense(64, activation='relu'))
model5.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
epochs = 10
steps_per_epoch160 = train_generator160.n // batch_size64
validation_steps160 = test_generator160.n // batch_size64

model5.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model5.fit_generator(train_generator160,
                     steps_per_epoch = steps_per_epoch160,
                     epochs=epochs,
                     workers=4,
                     validation_data=test_generator160,
                     validation_steps=validation_steps160)

analysis:  batch size 64 performed slightly better, but lack of regularization hurt the overall performance

solve: add batch normalization to each layer, and view results

## regularization: batch normalization w/ batch size 64

In [ ]:
image_size = 160
batch_size64 = 64

train_generator160 = datagen.flow_from_directory(
                train_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size64,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

test_generator160 = datagen.flow_from_directory(
                test_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size64,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')



In [ ]:
model6 = tf.keras.Sequential()

model6.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)))
model6.add(layers.MaxPooling2D((2, 2)))

model6.add(layers.Conv2D(64, (3, 3), activation='relu'))
model6.add(layers.BatchNormalization())
model6.add(layers.MaxPooling2D((2, 2)))

model6.add(layers.Conv2D(64, (3, 3), activation='relu'))
model6.add(layers.BatchNormalization())
model6.add(layers.MaxPooling2D((2, 2)))

model6.add(layers.Flatten())
model6.add(layers.Dense(64, activation='relu'))
model6.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
epochs = 10
steps_per_epoch160 = train_generator160.n // batch_size64
validation_steps160 = test_generator160.n // batch_size64

model6.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model6.fit_generator(train_generator160,
                     steps_per_epoch = steps_per_epoch160,
                     epochs=epochs,
                     workers=4,
                     validation_data=test_generator160,
                     validation_steps=validation_steps160)

## regularization: batch normalization w/ batch size 32

In [3]:
image_size = 160
batch_size = 32

train_generator160 = datagen.flow_from_directory(
                train_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

test_generator160 = datagen.flow_from_directory(
                test_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [4]:
model7 = tf.keras.Sequential()

model7.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)))
model7.add(layers.MaxPooling2D((2, 2)))

model7.add(layers.Conv2D(64, (3, 3), activation='relu'))
model7.add(layers.BatchNormalization())
model7.add(layers.MaxPooling2D((2, 2)))

model7.add(layers.Conv2D(64, (3, 3), activation='relu'))
model7.add(layers.BatchNormalization())
model7.add(layers.MaxPooling2D((2, 2)))

model7.add(layers.Flatten())
model7.add(layers.Dense(64, activation='relu'))
model7.add(layers.Dense(1, activation='sigmoid'))

W0414 14:26:52.197391 4722408896 deprecation.py:506] From /Users/kyledecember1/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
epochs = 5
steps_per_epoch160 = train_generator160.n // batch_size
validation_steps160 = test_generator160.n // batch_size

model7.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model7.fit_generator(train_generator160,
                     steps_per_epoch = steps_per_epoch160,
                     epochs=epochs,
                     workers=4,
                     validation_data=test_generator160,
                     validation_steps=validation_steps160)

W0414 14:26:53.204540 4722408896 deprecation.py:323] From /Users/kyledecember1/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
163/163 [==============================] - 266s 2s/step - loss: 0.2270 - acc: 0.9304 - val_loss: 0.9777 - val_acc: 0.6217
Epoch 2/5
163/163 [==============================] - 229s 1s/step - loss: 0.0756 - acc: 0.9751 - val_loss: 1.2767 - val_acc: 0.6234
Epoch 3/5
163/163 [==============================] - 226s 1s/step - loss: 0.0453 - acc: 0.9837 - val_loss: 1.1286 - val_acc: 0.6546
Epoch 4/5
163/163 [==============================] - 233s 1s/step - loss: 0.0283 - acc: 0.9898 - val_loss: 1.1076 - val_acc: 0.7664
Epoch 5/5
163/163 [==============================] - 207s 1s/step - loss: 0.0156 - acc: 0.9937 - val_loss: 3.9594 - val_acc: 0.6826


## adding dropout layers / increasing total layers of convolution

In [6]:
image_size = 160
batch_size = 32

train_generator160 = datagen.flow_from_directory(
                train_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

test_generator160 = datagen.flow_from_directory(
                test_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [7]:
model8 = tf.keras.Sequential()

model8.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)))
model8.add(layers.MaxPooling2D((2, 2)))

model8.add(layers.Conv2D(64, (3, 3), activation='relu'))
model8.add(layers.BatchNormalization())
model8.add(layers.MaxPooling2D((2, 2)))

model8.add(layers.Conv2D(128, (3, 3), activation='relu'))
model8.add(layers.BatchNormalization())
model8.add(layers.MaxPooling2D((2, 2)))

model8.add(layers.Conv2D(256, (3, 3), activation='relu'))
model8.add(layers.BatchNormalization())
model8.add(layers.MaxPooling2D((2, 2)))

model8.add(layers.Flatten())
model8.add(layers.Dense(256, activation='relu'))
model8.add(layers.Dropout(.3))
model8.add(layers.Dense(128, activation='relu'))
model8.add(layers.Dropout(.2))
model8.add(layers.Dense(64, activation='relu'))
model8.add(layers.Dropout(.1))

model8.add(layers.Dense(1, activation='sigmoid'))

In [8]:
epochs = 5
steps_per_epoch160 = train_generator160.n // batch_size
validation_steps160 = test_generator160.n // batch_size

model8.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model8.fit_generator(train_generator160,
                     steps_per_epoch = steps_per_epoch160,
                     epochs=epochs,
                     workers=4,
                     validation_data=test_generator160,
                     validation_steps=validation_steps160)

Epoch 1/5
163/163 [==============================] - 281s 2s/step - loss: 0.2128 - acc: 0.9231 - val_loss: 2.2559 - val_acc: 0.6266
Epoch 2/5
163/163 [==============================] - 273s 2s/step - loss: 0.1299 - acc: 0.9555 - val_loss: 4.1423 - val_acc: 0.6266
Epoch 3/5
163/163 [==============================] - 281s 2s/step - loss: 0.0967 - acc: 0.9703 - val_loss: 2.2651 - val_acc: 0.6414
Epoch 4/5
163/163 [==============================] - 317s 2s/step - loss: 0.0745 - acc: 0.9757 - val_loss: 0.3036 - val_acc: 0.8914
Epoch 5/5
163/163 [==============================] - 270s 2s/step - loss: 0.0586 - acc: 0.9835 - val_loss: 5.3896 - val_acc: 0.6447


#### analysis

every model beyond model 3 has only decreased validation accuracy

removing batch normalization from model 8 to investigate performance

## remove batch normalization

In [9]:
image_size = 160
batch_size = 32

train_generator160 = datagen.flow_from_directory(
                train_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

test_generator160 = datagen.flow_from_directory(
                test_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [10]:
model9 = tf.keras.Sequential()

model9.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)))
model9.add(layers.MaxPooling2D((2, 2)))

model9.add(layers.Conv2D(64, (3, 3), activation='relu'))
model9.add(layers.MaxPooling2D((2, 2)))

model9.add(layers.Conv2D(128, (3, 3), activation='relu'))
model9.add(layers.MaxPooling2D((2, 2)))

model9.add(layers.Conv2D(256, (3, 3), activation='relu'))
model9.add(layers.MaxPooling2D((2, 2)))

model9.add(layers.Flatten())
model9.add(layers.Dense(256, activation='relu'))
model9.add(layers.Dropout(.3))
model9.add(layers.Dense(128, activation='relu'))
model9.add(layers.Dropout(.2))
model9.add(layers.Dense(64, activation='relu'))
model9.add(layers.Dropout(.1))

model9.add(layers.Dense(1, activation='sigmoid'))

In [11]:
epochs = 5
steps_per_epoch160 = train_generator160.n // batch_size
validation_steps160 = test_generator160.n // batch_size

model9.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model9.fit_generator(train_generator160,
                     steps_per_epoch = steps_per_epoch160,
                     epochs=epochs,
                     workers=4,
                     validation_data=test_generator160,
                     validation_steps=validation_steps160)

Epoch 1/5
163/163 [==============================] - 201s 1s/step - loss: 0.4682 - acc: 0.7960 - val_loss: 0.3398 - val_acc: 0.8651
Epoch 2/5
163/163 [==============================] - 199s 1s/step - loss: 0.1990 - acc: 0.9224 - val_loss: 0.6866 - val_acc: 0.7763
Epoch 3/5
163/163 [==============================] - 223s 1s/step - loss: 0.1469 - acc: 0.9461 - val_loss: 0.6164 - val_acc: 0.7763
Epoch 4/5
163/163 [==============================] - 207s 1s/step - loss: 0.1094 - acc: 0.9601 - val_loss: 0.6047 - val_acc: 0.8224
Epoch 5/5
163/163 [==============================] - 208s 1s/step - loss: 0.0979 - acc: 0.9628 - val_loss: 0.8417 - val_acc: 0.7862


#### analysis

improved performance without Batch Normalization


## adding more dropout

In [14]:
image_size = 160
batch_size = 32

train_generator160 = datagen.flow_from_directory(
                train_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

test_generator160 = datagen.flow_from_directory(
                test_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [15]:
model10 = tf.keras.Sequential()

model10.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)))
model10.add(layers.MaxPooling2D((2, 2)))

model10.add(layers.Conv2D(64, (3, 3), activation='relu'))
model10.add(layers.MaxPooling2D((2, 2)))
model10.add(layers.Dropout(.2))

model10.add(layers.Conv2D(128, (3, 3), activation='relu'))
model10.add(layers.MaxPooling2D((2, 2)))
model10.add(layers.Dropout(.2))

model10.add(layers.Conv2D(256, (3, 3), activation='relu'))
model10.add(layers.MaxPooling2D((2, 2)))
model10.add(layers.Dropout(.2))

model10.add(layers.Flatten())
model10.add(layers.Dense(256, activation='relu'))
model10.add(layers.Dropout(.3))
model10.add(layers.Dense(128, activation='relu'))
model10.add(layers.Dropout(.2))
model10.add(layers.Dense(64, activation='relu'))
model10.add(layers.Dropout(.1))

model10.add(layers.Dense(1, activation='sigmoid'))

In [16]:
epochs = 5
steps_per_epoch160 = train_generator160.n // batch_size
validation_steps160 = test_generator160.n // batch_size

model10.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model10.fit_generator(train_generator160,
                     steps_per_epoch = steps_per_epoch160,
                     epochs=epochs,
                     workers=4,
                     validation_data=test_generator160,
                     validation_steps=validation_steps160)

Epoch 1/5
163/163 [==============================] - 288s 2s/step - loss: 0.5253 - acc: 0.7584 - val_loss: 0.4900 - val_acc: 0.8421
Epoch 2/5
163/163 [==============================] - 229s 1s/step - loss: 0.2897 - acc: 0.8806 - val_loss: 0.3709 - val_acc: 0.8553
Epoch 3/5
163/163 [==============================] - 227s 1s/step - loss: 0.2027 - acc: 0.9179 - val_loss: 0.5105 - val_acc: 0.7763
Epoch 4/5
163/163 [==============================] - 233s 1s/step - loss: 0.1551 - acc: 0.9406 - val_loss: 0.3815 - val_acc: 0.8372
Epoch 5/5
163/163 [==============================] - 225s 1s/step - loss: 0.1346 - acc: 0.9469 - val_loss: 0.4697 - val_acc: 0.7829


#### analysis

still overfitting to training data, increasing dropout rates

In [40]:
image_size = 160
batch_size = 32

train_generator160 = datagen.flow_from_directory(
                train_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary',
                shuffle=False)

test_generator160 = datagen.flow_from_directory(
                test_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary',
                shuffle=False)

Found 4986 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [4]:
model11 = tf.keras.Sequential()

model11.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)))
model11.add(layers.MaxPooling2D((2, 2)))

model11.add(layers.Conv2D(64, (3, 3), activation='relu'))
model11.add(layers.MaxPooling2D((2, 2)))
model11.add(layers.Dropout(.3))

model11.add(layers.Conv2D(128, (3, 3), activation='relu'))
model11.add(layers.MaxPooling2D((2, 2)))
model11.add(layers.Dropout(.4))

model11.add(layers.Conv2D(256, (3, 3), activation='relu'))
model11.add(layers.MaxPooling2D((2, 2)))
model11.add(layers.Dropout(.6))

model11.add(layers.Flatten())
model11.add(layers.Dense(256, activation='relu'))
model11.add(layers.Dropout(.6))
model11.add(layers.Dense(128, activation='relu'))
model11.add(layers.Dropout(.4))
model11.add(layers.Dense(64, activation='relu'))
model11.add(layers.Dropout(.2))

model11.add(layers.Dense(1, activation='sigmoid'))

W0416 10:24:30.513206 4538322368 deprecation.py:506] From /Users/kyledecember1/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0416 10:24:30.674054 4538322368 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0416 10:24:30.730453 4538322368 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


In [33]:
epochs = 5
steps_per_epoch160 = train_generator160.n // batch_size
validation_steps160 = test_generator160.n // batch_size

model11.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model11.fit(train_generator160,
                     steps_per_epoch = steps_per_epoch160,
                     epochs=epochs,
                     workers=4,
                     validation_data=test_generator160,
                     validation_steps=validation_steps160)

Epoch 1/5
  9/155 [>.............................] - ETA: 7:02 - loss: 0.2952 - acc: 0.9271

KeyboardInterrupt: 

In [62]:
import pickle
from sklearn import metrics

import os,sys
try:
    import lime
except:
    sys.path.append(os.path.join('..', '..')) # add the current directory
    import lime
from lime import lime_image

In [9]:
from tensorflow.keras.models import load_model
model11.save('nn_model.h5')  # creates a HDF5 file 'my_model.h5'
# del model  # deletes the existing model
# returns a compiled model
# identical to the previous one
model_new = load_model('nn_model.h5')

W0416 10:46:12.340668 4538322368 deprecation.py:506] From /Users/kyledecember1/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0416 10:46:12.341748 4538322368 deprecation.py:506] From /Users/kyledecember1/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0416 10:46:12.489216 4538322368 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Plea

In [41]:
preds = []
for x in model_new.predict_generator(test_generator160):
    for z in x:
        preds.append(np.round(z))
        
metrics.confusion_matrix(test_generator160.labels, preds)

array([[214,  20],
       [ 70, 320]])

In [17]:
final_test_generator = datagen.flow_from_directory(
                final_test_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary',
                shuffle=False)

Found 246 images belonging to 2 classes.


In [42]:
predictions2 = []
for x in model_new.predict_generator(final_test_generator):
    for z in x:
        predictions2.append(np.round(z))
        
metrics.confusion_matrix(final_test_generator.labels, predictions2)

array([[75, 55],
       [81, 35]])

In [45]:
!pip install scikit-image

     |████████████████████████████████| 30.4MB 1.1MB/s eta 0:00:01
     |████████████████████████████████| 4.3MB 29.6MB/s eta 0:00:01
     |████████████████████████████████| 3.3MB 39.9MB/s eta 0:00:01


In [46]:
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions
from skimage.io import imread
import matplotlib.pyplot as plt

In [63]:
explainer = lime_image.LimeImageExplainer()

In [64]:
explanation = explainer.explain_instance(train_generator160[0], model_new.predict, top_labels=5, hide_color=0, num_samples=1000)

AttributeError: 'tuple' object has no attribute 'shape'

In [59]:
img_path = 'person27_bacteria_135.jpeg'
img = image.load_img(img_path, target_size=(160, 160))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
img

array([[0.13224185],
       [0.15948376],
       [0.22123545],
       [0.17403579],
       [0.15958577],
       [0.17377862],
       [0.20354733],
       [0.17014503],
       [0.17615831],
       [0.15804839],
       [0.19419083],
       [0.19881377],
       [0.18041053],
       [0.18751505],
       [0.20761913],
       [0.22741958],
       [0.12730455],
       [0.16603217],
       [0.20559838],
       [0.12726372],
       [0.16384733],
       [0.22310248],
       [0.18498936],
       [0.18415824],
       [0.14804041],
       [0.1996808 ],
       [0.17181745],
       [0.14631048],
       [0.14830452],
       [0.24849802],
       [0.22717094],
       [0.20597667]], dtype=float32)

## analysis

performed better with higher levels of dropout

next steps:  
shift heavier weight dropout to end of network   
remove dropout from Convolution layers

In [3]:
image_size = 160
batch_size = 32

train_generator160 = datagen.flow_from_directory(
                train_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

test_generator160 = datagen.flow_from_directory(
                test_path,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

Found 4986 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [6]:
model11 = tf.keras.Sequential()

model11.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)))
model11.add(layers.MaxPooling2D((2, 2)))

model11.add(layers.Conv2D(64, (3, 3), activation='relu'))
model11.add(layers.MaxPooling2D((2, 2)))

model11.add(layers.Conv2D(128, (3, 3), activation='relu'))
model11.add(layers.MaxPooling2D((2, 2)))

model11.add(layers.Conv2D(256, (3, 3), activation='relu'))
model11.add(layers.MaxPooling2D((2, 2)))

model11.add(layers.Flatten())
model11.add(layers.Dense(256, activation='relu'))
model11.add(layers.Dropout(.6))
model11.add(layers.Dense(128, activation='relu'))
model11.add(layers.Dropout(.4))
model11.add(layers.Dense(64, activation='relu'))
model11.add(layers.Dropout(.2))

model11.add(layers.Dense(1, activation='sigmoid'))

W0415 13:54:01.555767 4652023232 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


In [7]:
epochs = 5
steps_per_epoch160 = train_generator160.n // batch_size
validation_steps160 = test_generator160.n // batch_size

model11.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model11.fit_generator(train_generator160,
                     steps_per_epoch = steps_per_epoch160,
                     epochs=epochs,
                     workers=4,
                     validation_data=test_generator160,
                     validation_steps=validation_steps160)

Epoch 1/5
155/155 [==============================] - 199s 1s/step - loss: 0.5482 - acc: 0.7539 - val_loss: 0.7950 - val_acc: 0.6217
Epoch 2/5
155/155 [==============================] - 184s 1s/step - loss: 0.3401 - acc: 0.8171 - val_loss: 0.6482 - val_acc: 0.7418
Epoch 3/5
155/155 [==============================] - 197s 1s/step - loss: 0.2194 - acc: 0.9160 - val_loss: 0.4331 - val_acc: 0.8487
Epoch 4/5
155/155 [==============================] - 199s 1s/step - loss: 0.1689 - acc: 0.9396 - val_loss: 0.8239 - val_acc: 0.7549
Epoch 5/5
155/155 [==============================] - 199s 1s/step - loss: 0.1390 - acc: 0.9516 - val_loss: 1.0411 - val_acc: 0.7451


## analysis

too overfit
increase dropout @ the dense layers

In [10]:
model12 = tf.keras.Sequential()

model12.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)))
model12.add(layers.MaxPooling2D((2, 2)))

model12.add(layers.Conv2D(64, (3, 3), activation='relu'))
model12.add(layers.MaxPooling2D((2, 2)))

model12.add(layers.Conv2D(128, (3, 3), activation='relu'))
model12.add(layers.MaxPooling2D((2, 2)))

model12.add(layers.Conv2D(256, (3, 3), activation='relu'))
model12.add(layers.MaxPooling2D((2, 2)))

model12.add(layers.Flatten())
model12.add(layers.Dense(256, activation='relu'))
model12.add(layers.Dropout(.8))
model12.add(layers.Dense(128, activation='relu'))
model12.add(layers.Dropout(.6))
model12.add(layers.Dense(64, activation='relu'))
model12.add(layers.Dropout(.4))

model12.add(layers.Dense(1, activation='sigmoid'))

W0415 14:24:04.050050 4652023232 nn_ops.py:4224] Large dropout rate: 0.8 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


In [11]:
epochs = 5
steps_per_epoch160 = train_generator160.n // batch_size
validation_steps160 = test_generator160.n // batch_size

model12.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model12.fit_generator(train_generator160,
                     steps_per_epoch = steps_per_epoch160,
                     epochs=epochs,
                     workers=4,
                     validation_data=test_generator160,
                     validation_steps=validation_steps160)

Epoch 1/5
155/155 [==============================] - 227s 1s/step - loss: 0.5930 - acc: 0.7350 - val_loss: 0.6278 - val_acc: 0.6217
Epoch 2/5
155/155 [==============================] - 235s 2s/step - loss: 0.4410 - acc: 0.7551 - val_loss: 0.4977 - val_acc: 0.6217
Epoch 3/5
155/155 [==============================] - 203s 1s/step - loss: 0.3135 - acc: 0.7719 - val_loss: 0.6032 - val_acc: 0.6382
Epoch 4/5
155/155 [==============================] - 182s 1s/step - loss: 0.2683 - acc: 0.8833 - val_loss: 0.5134 - val_acc: 0.7089
Epoch 5/5
155/155 [==============================] - 182s 1s/step - loss: 0.2503 - acc: 0.9158 - val_loss: 0.7992 - val_acc: 0.6793


## add batchnormalization to convolutional layers

In [12]:
model13 = tf.keras.Sequential()

model13.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)))
model13.add(layers.BatchNormalization())
model13.add(layers.MaxPooling2D((2, 2)))

model13.add(layers.Conv2D(64, (3, 3), activation='relu'))
model13.add(layers.BatchNormalization())
model13.add(layers.MaxPooling2D((2, 2)))

model13.add(layers.Conv2D(128, (3, 3), activation='relu'))
model13.add(layers.BatchNormalization())
model13.add(layers.MaxPooling2D((2, 2)))

model13.add(layers.Conv2D(256, (3, 3), activation='relu'))
model13.add(layers.BatchNormalization())
model13.add(layers.MaxPooling2D((2, 2)))

model13.add(layers.Flatten())
model12.add(layers.Dense(256, activation='relu'))
model13.add(layers.Dropout(.8))
model13.add(layers.Dense(128, activation='relu'))
model13.add(layers.Dropout(.6))
model13.add(layers.Dense(64, activation='relu'))
model13.add(layers.Dropout(.4))

model13.add(layers.Dense(1, activation='sigmoid'))

In [13]:
epochs = 5
steps_per_epoch160 = train_generator160.n // batch_size
validation_steps160 = test_generator160.n // batch_size

model13.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model13.fit_generator(train_generator160,
                     steps_per_epoch = steps_per_epoch160,
                     epochs=epochs,
                     workers=4,
                     validation_data=test_generator160,
                     validation_steps=validation_steps160)

Epoch 1/5
155/155 [==============================] - 342s 2s/step - loss: 0.8122 - acc: 0.7951 - val_loss: 4.3175 - val_acc: 0.6217
Epoch 2/5
155/155 [==============================] - 338s 2s/step - loss: 0.4050 - acc: 0.8864 - val_loss: 8.2578 - val_acc: 0.6217
Epoch 3/5
155/155 [==============================] - 376s 2s/step - loss: 0.3105 - acc: 0.9005 - val_loss: 6.9283 - val_acc: 0.6217
Epoch 4/5
155/155 [==============================] - 467s 3s/step - loss: 0.2820 - acc: 0.9162 - val_loss: 6.8764 - val_acc: 0.6234
Epoch 5/5
155/155 [==============================] - 364s 2s/step - loss: 0.2477 - acc: 0.9253 - val_loss: 4.2100 - val_acc: 0.6414


## reomove batch normalization
## increase learning rate

In [14]:
model14 = tf.keras.Sequential()

model14.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)))
model14.add(layers.MaxPooling2D((2, 2)))

model14.add(layers.Conv2D(64, (3, 3), activation='relu'))
model14.add(layers.MaxPooling2D((2, 2)))

model14.add(layers.Conv2D(128, (3, 3), activation='relu'))
model14.add(layers.MaxPooling2D((2, 2)))

model14.add(layers.Conv2D(256, (3, 3), activation='relu'))
model14.add(layers.MaxPooling2D((2, 2)))

model14.add(layers.Flatten())
model14.add(layers.Dense(256, activation='relu'))
model14.add(layers.Dropout(.8))
model14.add(layers.Dense(128, activation='relu'))
model14.add(layers.Dropout(.6))
model14.add(layers.Dense(64, activation='relu'))
model14.add(layers.Dropout(.4))

model14.add(layers.Dense(1, activation='sigmoid'))

In [15]:
epochs = 5
steps_per_epoch160 = train_generator160.n // batch_size
validation_steps160 = test_generator160.n // batch_size

model14.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model14.fit_generator(train_generator160,
                     steps_per_epoch = steps_per_epoch160,
                     epochs=epochs,
                     workers=4,
                     validation_data=test_generator160,
                     validation_steps=validation_steps160)

Epoch 1/5
155/155 [==============================] - 194s 1s/step - loss: 0.5430 - acc: 0.7453 - val_loss: 0.6052 - val_acc: 0.7582
Epoch 2/5
155/155 [==============================] - 193s 1s/step - loss: 0.3127 - acc: 0.8694 - val_loss: 0.6599 - val_acc: 0.8536
Epoch 3/5
155/155 [==============================] - 193s 1s/step - loss: 0.2047 - acc: 0.9279 - val_loss: 1.0452 - val_acc: 0.8043
Epoch 4/5
155/155 [==============================] - 202s 1s/step - loss: 0.1612 - acc: 0.9495 - val_loss: 1.7028 - val_acc: 0.7878
Epoch 5/5
155/155 [==============================] - 197s 1s/step - loss: 0.1445 - acc: 0.9554 - val_loss: 1.3855 - val_acc: 0.7434


## change optimizer to Adam

In [17]:
model15 = tf.keras.Sequential()

model15.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)))
model15.add(layers.MaxPooling2D((2, 2)))

model15.add(layers.Conv2D(64, (3, 3), activation='relu'))
model15.add(layers.MaxPooling2D((2, 2)))

model15.add(layers.Conv2D(128, (3, 3), activation='relu'))
model15.add(layers.MaxPooling2D((2, 2)))

model15.add(layers.Conv2D(256, (3, 3), activation='relu'))
model15.add(layers.MaxPooling2D((2, 2)))

model15.add(layers.Flatten())
model15.add(layers.Dense(256, activation='relu'))
model15.add(layers.Dropout(.6))
model15.add(layers.Dense(128, activation='relu'))
model15.add(layers.Dropout(.3))
model15.add(layers.Dense(64, activation='relu'))
model15.add(layers.Dropout(.1))

model15.add(layers.Dense(1, activation='sigmoid'))

In [19]:
epochs = 10
steps_per_epoch160 = train_generator160.n // batch_size
validation_steps160 = test_generator160.n // batch_size

model15.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model15.fit_generator(train_generator160,
                     steps_per_epoch = steps_per_epoch160,
                     epochs=epochs,
                     workers=4,
                     validation_data=test_generator160,
                     validation_steps=validation_steps160)

Epoch 1/10
155/155 [==============================] - 183s 1s/step - loss: 0.3946 - acc: 0.8325 - val_loss: 0.8341 - val_acc: 0.7730
Epoch 2/10
155/155 [==============================] - 178s 1s/step - loss: 0.1554 - acc: 0.9421 - val_loss: 0.7726 - val_acc: 0.8141
Epoch 3/10
155/155 [==============================] - 175s 1s/step - loss: 0.1178 - acc: 0.9588 - val_loss: 1.1350 - val_acc: 0.7648
Epoch 4/10
155/155 [==============================] - 176s 1s/step - loss: 0.0951 - acc: 0.9661 - val_loss: 1.4714 - val_acc: 0.7862
Epoch 5/10
155/155 [==============================] - 174s 1s/step - loss: 0.0852 - acc: 0.9734 - val_loss: 1.0854 - val_acc: 0.7566
Epoch 6/10
155/155 [==============================] - 177s 1s/step - loss: 0.0714 - acc: 0.9715 - val_loss: 1.0796 - val_acc: 0.7878
Epoch 7/10
155/155 [==============================] - 175s 1s/step - loss: 0.0557 - acc: 0.9804 - val_loss: 1.2744 - val_acc: 0.8141
Epoch 8/10
155/155 [==============================] - 175s 1s/step - 

In [20]:
import pickle
